In [12]:
import h5py
filenameCamera = '/media/is-lab/2E82556B82553913/SP/camera/2016-01-31--19-19-25.h5'
fCamera = h5py.File(filenameCamera, 'r')

list (fCamera)

# List all groups
#print("Keys: %s" % f.keys())
#a_group_key = list(f.keys())[0]

# Get the data
#data = list(f[a_group_key])
#print (data)

['X']

In [13]:
fCamera['X'].shape

(19731, 3, 160, 320)

In [14]:
import h5py
filenameLog = '/media/is-lab/2E82556B82553913/SP/log/2016-01-31--19-19-25.h5'
fLog = h5py.File(filenameLog, 'r')

list (fLog)


['UN_D_cam1_ptr',
 'UN_D_cam2_ptr',
 'UN_D_camphone_ptr',
 'UN_D_lidar_ptr',
 'UN_D_radar_msg',
 'UN_D_rawgps',
 'UN_T_cam1_ptr',
 'UN_T_cam2_ptr',
 'UN_T_camphone_ptr',
 'UN_T_lidar_ptr',
 'UN_T_radar_msg',
 'UN_T_rawgps',
 'blinker',
 'brake',
 'brake_computer',
 'brake_user',
 'cam1_ptr',
 'cam2_ptr',
 'camphone_ptr',
 'car_accel',
 'fiber_accel',
 'fiber_compass',
 'fiber_compass_x',
 'fiber_compass_y',
 'fiber_compass_z',
 'fiber_gyro',
 'fiber_temperature',
 'gas',
 'gear_choice',
 'gps_1_fix',
 'gps_1_pos',
 'gps_1_vel',
 'gps_2_fix',
 'gps_2_pos',
 'gps_2_vel',
 'idx',
 'imu_accel',
 'imu_compass',
 'imu_gyro',
 'rpm',
 'rpm_post_torque',
 'selfdrive',
 'speed',
 'speed_abs',
 'speed_fl',
 'speed_fr',
 'speed_rl',
 'speed_rr',
 'standstill',
 'steering_angle',
 'steering_torque',
 'times',
 'velodyne_gps',
 'velodyne_heading',
 'velodyne_imu']

In [3]:

a = fLog['cam1_ptr']
#len([i for i, e in enumerate(a) if e != 0])
a.shape

(98719,)

In [4]:
a1 = list(a)
#print (a1)
#print(a[3000:4000])
#a.index("621")
#print(a[294916])

In [5]:
a1.index(621)

3141

In [ ]:
fLog['speed'][3148]

In [6]:
prev = 0
k = 0
sum = 0
l = 0
m = list()
for i in a:
    if i!=prev:
        m.append(sum/l)
        l = 1
        sum = fLog['speed'][k];
    else:
        sum = sum + fLog['speed'][k]
        l = l+1
    prev = i
    k = k+1
    
print(len(m))

19731


In [ ]:
#len(m)

In [ ]:
image = fCamera['X'][4091,:,:,:]
print(image.shape)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

def rgb2gray(rgb):
    r, g, b = rgb[0,:,:], rgb[1,:,:], rgb[2,:,:]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    #return np.dot(rgb[:3,:,:], [0.299, 0.587, 0.114])
    return gray
     
from scipy.misc import toimage
from scipy.misc import imshow
imshow(image)

gray = rgb2gray(image)  
imshow(gray)

In [8]:
x = fCamera['X'].value
y = np.asarray(m)
#x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=100)


/home/is-lab/.local/lib/python3.5/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [11]:
#c = np.c_[x.reshape(len(x), -1), y.reshape(len(y), -1)]
#np.random.shuffle(c)

NameError: name 'c' is not defined

In [9]:
fCamera.close()
fLog.close()
#x = c[:, :x.size//len(x)].reshape(x.shape)
#y = c[:, x.size//len(y):].reshape(y.shape)
print(x.shape, y.shape)

NameError: name 'c' is not defined

In [ ]:
x_train = x[0:16000, :, :, :]
x_test = x[16000:19731, :, :, :]
y_train = y[0:16000]
y_test = y[16000:19731]
print (x_train.shape)
print(x_test.shape)
print(len(y_train))
print(len(y_test))

In [ ]:
x_train = np.rollaxis(x_train, 1, 4)
x_test = np.rollaxis(x_test, 1, 4)
print(x_train.shape)
x_test.shape

In [ ]:
#x_train = x_train  / 255.0
#x_test = x_test / 255.0
#memory error

In [ ]:
import tensorflow as tf
def multitask_loss(y_true, y_pred):
    loss1 = tf.losses.mean_squared_error(y_true, y_pred)

    jointLoss = loss1
    return jointLoss
    

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Activation

batch_size = 64
num_classes = 5
epochs = 25

# input image dimensions
img_rows, img_cols = 160, 320
channels = 3

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding='same',input_shape=(img_rows, img_cols, channels)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='normal'))
#model.add(Activation('sigmoid'))


model.compile(loss=multitask_loss,
              optimizer='adam')

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
from keras.models import model_from_json

# serialize model to JSON
model_json = model.to_json()
with open("model10epochsActualshuffled.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model10epochsActualshuffled.h5")
print("Saved model to disk")


In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_predict = model.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test, y_predict)

In [ ]:
y_test[3000:3730]